# 03. Self-Service Help Desk



In [1]:
#Install all related packages. If you find additional packages missing, please follow the same technique.
#If you are not using anaconda, then use pip to install the same packages

import sys
!conda install --yes --prefix {sys.prefix} pandas tensorflow scikit-learn gensim

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



## 03.04. Building a Document Vector

In [2]:
from collections import defaultdict
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np
import os
import pandas as pd

#Read the input CSV into a Pandas dataframe
helpdesk_data = pd.read_csv("helpdesk_dataset.csv")

print("HelpDesk Data: ")
print(helpdesk_data.head())



HelpDesk Data: 
                                            Question  \
0              My Mac does not boot, what can I do ?   
1                Can Mac Air get infected by a Virus   
2   My Mac is having boot problems, how do I fix it?   
3                 Do I need an anti virus on my Mac?   
4  I have trouble connecting my monitor to my Mac...   

                   LinkToAnswer  
0  http://faq/mac-does-not-boot  
1     http://faq/mac-book-virus  
2  http://faq/mac-does-not-boot  
3     http://faq/mac-book-virus  
4  http://faq/mac-monitor-setup  


In [3]:
#Extract the Question column 
documents = helpdesk_data["Question"]

#Function to cleanse document
def process_document(document):

    #Remove stopwords, convert to lower case and remove "?" character
    cleaned_document = remove_stopwords(document.lower()).replace("?","")  
    return cleaned_document.split()

#Create a document vector
doc_vectors=[process_document(document)
             for document in documents]

#Print the document and the corresponding document vector to compare
print(documents[1])
print(doc_vectors[1])

Can Mac Air get infected by a Virus
['mac', 'air', 'infected', 'virus']


## 03.05. Creating the LSI Model

In [4]:
#Create the dictionary
dictionary = corpora.Dictionary(doc_vectors)

print("Dictionary created :")
dictionary.token2id


Dictionary created :


{'boot,': 0,
 'mac': 1,
 'air': 2,
 'infected': 3,
 'virus': 4,
 'boot': 5,
 'fix': 6,
 'having': 7,
 'it': 8,
 'problems,': 9,
 'anti': 10,
 'need': 11,
 'connecting': 12,
 'help': 13,
 'mac.': 14,
 'monitor': 15,
 'trouble': 16,
 'boots,': 17,
 'error': 18,
 'shows': 19,
 'software': 20,
 'unsupporterd': 21,
 'connected': 22,
 'proper': 23,
 'resolution': 24,
 'flicker': 25,
 'monitor.': 26,
 'hdmi': 27,
 'use': 28,
 'connect': 29,
 'monitors': 30}

In [5]:
#Create a corpus
corpus = [dictionary.doc2bow(doc_vector) 
          for doc_vector in doc_vectors]

#Review the corpus generated
print(doc_vectors[1])
print(corpus[1])

['mac', 'air', 'infected', 'virus']
[(1, 1), (2, 1), (3, 1), (4, 1)]


In [6]:
#Build the LSI Model
from gensim import models,similarities

#Create the model
lsi = models.LsiModel(corpus, id2word=dictionary)

#Create a similarity Index
index = similarities.MatrixSimilarity(lsi[corpus])

for similarities in index:
    print(similarities)

[ 1.0000000e+00  3.5355341e-01  2.8867516e-01  3.5355341e-01
  2.5866751e-08  2.8867513e-01 -2.9364895e-08  2.8446223e-10
  3.5355338e-01  4.0824834e-01]
[ 3.5355341e-01  9.9999994e-01  2.0412415e-01  5.0000000e-01
  2.2450529e-08  2.0412415e-01 -3.7067536e-09  2.8867513e-01
  2.5000003e-01  2.8867513e-01]
[ 2.8867516e-01  2.0412415e-01  9.9999994e-01  2.0412415e-01
 -1.8794251e-08  1.6666667e-01  3.0860671e-01  1.0505437e-08
  2.0412411e-01  2.3570226e-01]
[ 3.5355341e-01  5.0000000e-01  2.0412415e-01  1.0000000e+00
  2.2450529e-08  2.0412415e-01 -3.7067536e-09  2.8867513e-01
  2.5000003e-01  2.8867513e-01]
[ 2.5866751e-08  2.2450529e-08 -1.8794251e-08  2.2450529e-08
  1.0000000e+00  2.2861792e-08  3.3806166e-01 -4.6973598e-09
  2.2360680e-01 -4.1850909e-10]
[ 2.8867513e-01  2.0412415e-01  1.6666667e-01  2.0412415e-01
  2.2861792e-08  1.0000001e+00 -3.3151069e-09 -8.8221241e-10
  2.0412418e-01  2.3570229e-01]
[-2.9364895e-08 -3.7067536e-09  3.0860671e-01 -3.7067536e-09
  3.3806166e-01

## 03.06. Recommending FAQs

In [7]:

question = "I have boot problems in my Mac"

#Pre Process the Question 
question_corpus = dictionary.doc2bow(process_document(question))
print("Question translated to :", question_corpus)

#Create an LSI Representation
vec_lsi = lsi[question_corpus]  

#Find similarity of the question with existing documents
sims = index[vec_lsi]  
print("Similarity scores :",list(enumerate(sims)))

Question translated to : [(1, 1), (5, 1)]
Similarity scores : [(0, 0.67856914), (1, 0.47982085), (2, 0.7835442), (3, 0.47982085), (4, -1.4901161e-08), (5, 0.39177215), (6, -5.2154064e-08), (7, -7.450581e-09), (8, 0.47982085), (9, 0.55404943)]


In [8]:
#Find the corresponding FAQ Link

#sort an array in reverse order and get indexes
matches=np.argsort(sims)[::-1] 
print("Sorted Document index :", matches)

print("\n", "-"*60, "\n")
for i in matches:
    print(sims[i], " -> ", helpdesk_data.iloc[i]["Question"])

print("\n", "-"*60, "\n")
print("Recommended FAQ :" , helpdesk_data.iloc[matches[0]]["LinkToAnswer"] )

Sorted Document index : [2 0 9 8 3 1 5 7 4 6]

 ------------------------------------------------------------ 

0.7835442  ->  My Mac is having boot problems, how do I fix it?
0.67856914  ->  My Mac does not boot, what can I do ?
0.55404943  ->  Can I connect two monitors to my Mac?
0.47982085  ->  Can I use a HDMI monitor with my Mac?
0.47982085  ->  Do I need an anti virus on my Mac?
0.47982085  ->  Can Mac Air get infected by a Virus
0.39177215  ->  When my Mac boots, it shows an unsupporterd software error
-7.450581e-09  ->  I see a flicker in my monitor. Is that a virus?
-1.4901161e-08  ->  I have trouble connecting my monitor to my Mac. Can you please help?
-5.2154064e-08  ->  My Monitor does not show in proper resolution when connected to my Mac. How do I fix it?

 ------------------------------------------------------------ 

Recommended FAQ : http://faq/mac-does-not-boot
